## Get data from APIs

In [7]:
import pandas as pd
import requests
import json

### 1. /v4/gezocht
Search APB’s. Use consecutive methods to retrieve APB’s of a specialized type. Results are sorted on
descending date (newest first).

In [11]:
base_url = "https://api.politie.nl/v4/"
# The url filed list, you can pull more data according to this list
field_list ={
    "gezocht": "opsporingsberichten", 
    "politiebureaus/all": "politiebureaus", 
    "gezocht/gezochtepersonen" : "opsporingsberichten", 
    "gezocht/nationaleopsporingslijst" : "opsporingsberichten",
    "gezocht/eigenaargezocht": "opsporingsberichten",
    "gezocht/dossiers": "opsporingsberichten",
    "vermist" :'vermisten',
    "vermist/vermistevolwassenen":"vermisten",
    "vermist/vermistekinderen":"vermisten"
}

def get_data(target,target_field,limit=10):
    '''
    Retrive the data from a paginated API. 
    Args:
        target (str): The target URL field.
        target_field (str): The information field of the target URL.
        limit (int, optional): The maximum number of records to retrieve per request (default is 10).
    
    Returns:
        pandas.DataFrame: A DataFrame containing the retrieved data.
    '''
    offset = 0
    data_frames = []
    while True:
        response = requests.get(base_url + str(target),params={ "offset": offset})
        
        if response.status_code == 200:
            data = response.json()
            d = pd.DataFrame(data.get(str(target_field), []))  #
            data_frames.append(d)
            offset += limit
        elif response.status_code == 204:
            print("No results found.")
            break
        else:
            print("Error:", response.status_code)
            break
    return pd.concat(data_frames, ignore_index=True)

def save_data(target,field):
    '''
    Save data to a CSV file.
    Args:
        data (pandas.DataFrame): The DataFrame to be saved.
        field (str): The name of the field used to construct the CSV file name.
    Returns:
        None
    '''
    csv_filename = f"../data/daily_updated/{field}_10_6.csv"
    target.to_csv(csv_filename, index=False)
    return

In [13]:
fugitives = get_data("gezocht","opsporingsberichten")
save_data(fugitives,"1_fugitives")

police_stations = get_data("politiebureaus/all","politiebureaus",100)
save_data(police_stations,"4_police_stations")

missing_person = get_data("vermist","vermisten")
save_data(missing_person,"11_missing_person")

most_wanted = get_data("gezocht/nationaleopsporingslijst","opsporingsberichten")
save_data(most_wanted, "8_most_wanted")

No results found.
No results found.
No results found.
No results found.


In [14]:
unidentified_person = get_data("vermist/ongeidentificeerd","ongeidentificeerden")
save_data(unidentified_person,"14_unifentified_person")

No results found.
